# Feature Pipeline - Datos Históricos Completos (Semanas Completas)

> **OBJETIVO**: Este notebook replica la lógica del pipeline de datos pero utiliza la función `load_raw_data_historico` para obtener un conjunto completo de datos, asegurando además que todas las semanas son completas (de lunes a domingo).

## El enfoque:
1. Usar `load_raw_data_historico` que internamente utiliza `descargar_datos_bigquery_histórico`
2. Obtener datos completos desde ambas tablas de BigQuery
3. **Asegurar que los datos terminan en un domingo** (semana completa) usando el nuevo parámetro `usar_semana_completa=True`
4. Procesar y transformar los datos para el Feature Store, incluyendo solo semanas que tengan los 7 días
5. Preparar las series temporales para el entrenamiento de modelos

In [11]:
# 1. Imports

import sys
import os

# Añadir directorio raíz al path para importar correctamente los módulos
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from datetime import datetime, timedelta, date
from pathlib import Path
from google.cloud import bigquery
import pandas as pd
import numpy as np
from src import config
import hopsworks

# Importamos las funciones que necesitaremos
from src.data_utils import descargar_datos_bigquery_histórico, transformar_a_series_temporales, load_raw_data_historico


In [7]:
# Descargar datos históricos completos usando la función load_raw_data_historico
print("Iniciando carga de datos históricos completos...")

# Usar el nuevo parámetro usar_semana_completa para asegurar que las fechas finales terminan en domingo
df_raw = load_raw_data_historico(
    descargar_bq=True, 
    usar_semana_completa=True,
    fecha_inicio=None,  # Fecha inicial en formato YYYY-MM-DD
    fecha_fin=None      # Fecha final en formato YYYY-MM-DD
)

Iniciando carga de datos históricos completos...
Descargando datos históricos completos desde BigQuery
Iniciando conexión con BigQuery...
Conexión establecida.
Descargando datos de fleca-del-port.varios.raw_data_bq_forecasting_20250630 ...
Conexión establecida.
Descargando datos de fleca-del-port.varios.raw_data_bq_forecasting_20250630 ...


c:\Workspace\mlops_fleca_project\.venv\lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Filas descargadas de la primera tabla: 337353
Descargando datos de fleca-del-port.fleca_ventas_dia.t_facturas_dia_extendida_2023 ...
Filas descargadas de la segunda tabla: 21276
Total de filas tras concatenar: 358629
Guardando archivo en C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250817.parquet ...
Archivo guardado correctamente.
Filas descargadas de la segunda tabla: 21276
Total de filas tras concatenar: 358629
Guardando archivo en C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250817.parquet ...
Archivo guardado correctamente.
Ajustando fecha final a último domingo completo: 2025-08-10
Filtrando datos hasta: 2025-08-10 00:00:00
Ajustando fecha final a último domingo completo: 2025-08-10
Filtrando datos hasta: 2025-08-10 00:00:00


In [8]:
# 4. Transformar a series temporales semanales solo para la familia BOLLERIA
df_ts = transformar_a_series_temporales(df_raw, familia="BOLLERIA")
print('Series temporales generadas:', df_ts.shape)
print(df_ts.head())

Series temporales generadas: (132, 8)
   year  week   familia  base_imponible  is_summer_peak  is_easter  \
0  2023     1  BOLLERIA          825.11               0          0   
1  2023     2  BOLLERIA          658.40               0          0   
2  2023     3  BOLLERIA          741.40               0          0   
3  2023     4  BOLLERIA          653.64               0          0   
4  2023     5  BOLLERIA          680.46               0          0   

   dias_semana week_start  
0            7 2023-01-02  
1            7 2023-01-09  
2            7 2023-01-16  
3            7 2023-01-23  
4            7 2023-01-30  


In [9]:

# Ajustar tipos para coincidir con el schema del Feature Group histórico
df_ts['year'] = df_ts['year'].astype('int64')  # bigint
df_ts['week'] = df_ts['week'].astype('int64')  # bigint
df_ts['familia'] = df_ts['familia'].astype('string')  # string
df_ts['base_imponible'] = df_ts['base_imponible'].astype('float64')  # double
df_ts['is_summer_peak'] = df_ts['is_summer_peak'].astype('int32')  # int
df_ts['is_easter'] = df_ts['is_easter'].astype('int64')  # bigint
df_ts['week_start'] = pd.to_datetime(df_ts['week_start'])  # timestamp

print(df_ts.dtypes)
print(df_ts.head())

year                       int64
week                       int64
familia           string[python]
base_imponible           float64
is_summer_peak             int32
is_easter                  int64
dias_semana                int64
week_start        datetime64[ns]
dtype: object
   year  week   familia  base_imponible  is_summer_peak  is_easter  \
0  2023     1  BOLLERIA          825.11               0          0   
1  2023     2  BOLLERIA          658.40               0          0   
2  2023     3  BOLLERIA          741.40               0          0   
3  2023     4  BOLLERIA          653.64               0          0   
4  2023     5  BOLLERIA          680.46               0          0   

   dias_semana week_start  
0            7 2023-01-02  
1            7 2023-01-09  
2            7 2023-01-16  
3            7 2023-01-23  
4            7 2023-01-30  


In [12]:
# 5. Conectar a hopsworks
project = hopsworks.login(
    api_key_value=config.HOPSWORKS_API_KEY, 
    project=config.HOPSWORKS_PROJECT_NAME)

# Conectar al feature store
feature_store = project.get_feature_store()

# Conectar al Feature Group histórico
try:
    feature_group = feature_store.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=config.FEATURE_GROUP_VERSION,
        
    )
    if feature_group is None:
        raise Exception("El Feature Group histórico no existe o el nombre/version no coinciden exactamente. Verifica en Hopsworks.")
except Exception as e:
    print(f"Error al crear/conectar el Feature Group: {e}")

2025-08-17 19:56:46,524 INFO: Initializing external client
2025-08-17 19:56:46,524 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-17 19:56:46,524 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-17 19:56:48,009 INFO: Python Engine initialized.
2025-08-17 19:56:48,009 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272


In [13]:
# Solo para cuando estemos seguros que los datos están correctos
subir_a_hopsworks = False  # Cambiar a True cuando quieras subir los datos

if subir_a_hopsworks:
    # Importar módulos necesarios para Hopsworks
    import hopsworks
    from src import config
    
    # Conectar a Hopsworks
    project = hopsworks.login(
        api_key_value=config.HOPSWORKS_API_KEY, 
        project=config.HOPSWORKS_PROJECT_NAME)

    # Conectar al feature store
    feature_store = project.get_feature_store()

    # Conectar al Feature Group histórico
    try:
        feature_group = feature_store.get_feature_group(
            name=config.FEATURE_GROUP_NAME,
            version=config.FEATURE_GROUP_VERSION,
        )
        if feature_group is None:
            raise Exception("El Feature Group histórico no existe o el nombre/version no coinciden exactamente. Verifica en Hopsworks.")
        
        # Insertar los datos en el Feature Group
        print("Subiendo datos a Hopsworks...")
        feature_group.insert(
            df_ts,
            write_options={'wait_for_job': True}
        )
        print("¡Datos subidos con éxito a Hopsworks!")
    except Exception as e:
        print(f"Error al conectar o subir datos a Hopsworks: {e}")
else:
    print("Subida a Hopsworks desactivada.")
    print("Cambia 'subir_a_hopsworks = True' cuando quieras subir los datos.")

Subida a Hopsworks desactivada.
Cambia 'subir_a_hopsworks = True' cuando quieras subir los datos.


In [ ]:
# Insertar los datos en el Feature Group
feature_group.insert(
    df_ts,
    write_options={'wait_for_job': True}
)